In [39]:
import json
import glob
import pickle
import numpy as np
import MeCab

In [40]:
def corpusGenerator(fplist,train=False):
    results = []
    add = results.append
    for fp in fplist:
        data= json.load(open(fp))
        logs = data['turns']
        for i in range(1,len(logs),2):
            if train:
                annt = [a["breakdown"] for a in logs[i+1]["annotations"]]
                for an in annt:
                    probDist = list(map(int,[an=="O",an=="T",an=="X"]))
                    add((logs[i]["utterance"],logs[i+1]["utterance"],np.array(probDist)))
            else:
                annt = [a["breakdown"] for a in logs[i+1]["annotations"]]
                sm = len(annt)
                probDist = (annt.count("O")/sm,annt.count("T")/sm,annt.count("X")/sm)
                add((logs[i]["utterance"],logs[i+1]["utterance"],np.array(probDist)))
    return results

In [41]:
def mFilter(text,m):
    tmp = [word for word in m.parse(text).strip().split("\n")[:-1]]
    tmp1 = list(map(str.split,tmp))
    tmp2 = list(filter(lambda x: "名詞" not in x[1],tmp1))
    tmp3 = list(map(lambda x:x[0],tmp2))
    return tmp3

def tokenize(data,clustering=False):
    if clustering:
        m = MeCab.Tagger()
        return [(mFilter(user,m),mFilter(system,m),probDist) for user,system,probDist in data]
    else:
        m = MeCab.Tagger("-Owakati")
        return [(m.parse(user).strip().split(),m.parse(system).strip().split(),probDist) for user,system,probDist in data]

In [42]:
def w2vtrain():
    rawCorpus = "" #wordEmbedding学習用
    for data in tokenized_trainData:
        rawCorpus += "\n" + " ".join(data[0])
        rawCorpus += "\n" + " ".join(data[1])
    rawCorpus = rawCorpus.strip()
    open("rawCorpus.txt","w").write(rawCorpus)
    sentences = word2vec.LineSentence("rawCorpus.txt")
    w2v = word2vec.Word2Vec(sentences,window=1)
    w2v.save("w2v.model")
    return w2v

In [43]:
def vectorize(tokenizedData):
    g = lambda x: list(filter(lambda word:word in w2v.wv.vocab.keys(), x))
    f = lambda x: np.array([w2v.wv[word] for word in x])
    return np.array([(f(g(user)),f(g(system)),probDist) for user,system,probDist in tokenizedData])

In [44]:
from gensim.models import word2vec
#w2vpath='../BreakDialogue/word2vec.gensim.model'
w2vpath='./w2v.model'
try:
    w2v = word2vec.Word2Vec.load(w2vpath)
except:
    w2v = w2vtrain()

In [59]:

fp_rest = glob.glob('../projectnextnlp-chat-dialogue-corpus/json/rest1046/*.json')
fp_rest.extend (glob.glob('./DCM/*.json'))
fp_rest.extend(glob.glob('./DIT/*.json'))
fp_rest.extend(glob.glob('./IRS/*.json'))
fp_rest.extend(glob.glob('./dev/*.json'))
fp_init = glob.glob('../projectnextnlp-chat-dialogue-corpus/json/init100/*.json')
trainData = corpusGenerator(fp_rest,False)
validData = corpusGenerator(fp_init)
tokenized_trainData = tokenize(trainData)
tokenized_validData = tokenize(validData)
vectorized_trainData = vectorize(tokenized_trainData)
vectorized_validData = vectorize(tokenized_validData)

tmp = """
with open('corpus.pickle', mode='wb') as f:
    pickle.dump({
        "raw":{"train":trainData,"valid":validData},
        "tokenized":{"train":tokenized_trainData,"valid":tokenized_validData},
        "vectorized":{"train":vectorized_trainData,"valid":vectorized_validData},
    },f)
"""

In [64]:
from functools import reduce

In [70]:
M = reduce(lambda a,b: a if a> b else b,map(lambda x: max(len(x[0]),len(x[1])),tokenized_trainData))
m = reduce(lambda a,b: a if a< b else b,map(lambda x: max(len(x[0]),len(x[1])),tokenized_trainData))

1